# Working with Raster Bands

## Preparing Your Workspace

### Option 1: (recommended) Run in Google Colab
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevinlacaille/presentations/blob/main/scipy2024/2_exif.ipynb)

### Option 2: Run local Jupyter instance
You can also choose to open this Notebook in your own local Jupyter instance.

**Prerequisites**

- Install: rasterio, exiftool
- Download data

In [13]:
!pip install rasterio
!pip install PyExifTool
!apt-get install -y exiftool
!wget https://raw.githubusercontent.com/kevinlacaille/presentations/main/scipy2024/data/presentation/8928dec4ddbffff/DJI_0876.JPG

In [14]:
import os

image_path = "/content/DJI_0876.JPG" if os.path.exists(
    "/content/DJI_0876.JPG"
) else "data/presentation/8928dec4ddbffff/DJI_0876.JPG"

Import the image using Rasterio and reads the entire image (all 3 bands, red, green, blue) into a multi-dimensional array.

In [15]:
import rasterio

# import the image
with rasterio.open(image_path) as src:
    # read the entire image
    img = src.read()

print(img.shape)

/Users/kevinlacaille/.pyenv/versions/3.11.4/envs/scipy2024-env/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


(3, 3000, 4000)


In [16]:
import exiftool
import json

Extract the metadata from the image using ExifTool and convert it into a useable format (JSON) format for easy manipulation and analysis.

In [17]:
# Extract the metadata from the image
with exiftool.ExifTool() as et:
    metadata = json.loads(et.execute(b'-j', image_path))

Print the extracted metadata from the image, providing detailed information about the image and its properties. Here is a summary of key attributes:

- **File Information**: The image file name is `DJI_0876.JPG`, with a size of approximately 5.3 MB.
- **Image Details**: The image dimensions are 4000x3000 pixels, with an 8-bit depth and 3 color components. The camera model is `DJI FC3682`.
- **EXIF Data**: The exposure time is 0.000625 seconds (1/1600s), with an aperture value of 1.7 and ISO 100. The image was captured on April 20, 2024, at 11:48:07.
- **GPS Coordinates**: The image includes GPS metadata, pinpointing the location at latitude 49.2886° N and longitude 122.8356° W, with an altitude of 108.7 meters, relative to the takeoff location.
- **Additional Data**: The metadata also contains various details about the camera settings, including exposure program, metering mode, and focal length. The gimbal and flight details such as pitch, yaw, and roll angles are also recorded.

In [18]:
# Print the metadata json
print(json.dumps(metadata, indent=4))

[
    {
        "SourceFile": "data/presentation/8928dec4ddbffff/DJI_0876.JPG",
        "ExifTool:ExifToolVersion": 12.6,
        "ExifTool:Warning": "[minor] Possibly incorrect maker notes offsets (fix by 1783?)",
        "File:FileName": "DJI_0876.JPG",
        "File:Directory": "data/presentation/8928dec4ddbffff",
        "File:FileSize": 5257518,
        "File:FileModifyDate": "2024:06:14 03:25:40-07:00",
        "File:FileAccessDate": "2024:07:07 21:48:20-07:00",
        "File:FileInodeChangeDate": "2024:07:06 23:17:08-07:00",
        "File:FilePermissions": 100664,
        "File:FileType": "JPEG",
        "File:FileTypeExtension": "JPG",
        "File:MIMEType": "image/jpeg",
        "File:ExifByteOrder": "II",
        "File:ImageWidth": 4000,
        "File:ImageHeight": 3000,
        "File:EncodingProcess": 0,
        "File:BitsPerSample": 8,
        "File:ColorComponents": 3,
        "File:YCbCrSubSampling": "2 1",
        "EXIF:ImageDescription": "DCIM\\108MEDIA\\DJI_0876.JPG"

Set relevant metadata to variables for future analysis.

In [19]:
# Extract the GPS Altitude
altitude = float(metadata[0].get("XMP:RelativeAltitude"))
focal_length = metadata[0].get("EXIF:FocalLength")  # in mm
# Size of pixel = sensor width (m) / image width (px)
image_width = metadata[0].get("File:ImageWidth")
pixel_pitch = 6.17e-3 / image_width  # sensor width known to be 1/2.3” = 6.17mm

print("Altitude: ", altitude, "meters")
print("Focal Length: ", focal_length, "mm")
print("Pixel Pitch: ", pixel_pitch, "meters / px")

Altitude:  108.7 meters
Focal Length:  6.72 mm
Pixel Pitch:  1.5425e-06 meters / px


Ground Sampling Distance (GSD) is a measure of the spatial resolution of an image, representing the real-world size of one pixel in the image. Namely, how large a pixel is projected on the ground. It is calculated using the formula:

$$ \text{GSD} = \frac{\text{altitude} \times \text{pixel pitch}}{\text{focal length}} $$

- **Altitude**: The height at which the image was captured above the ground.
- **Pixel Pitch**: The physical distance between the centers of two adjacent pixels on the camera sensor.
- **Focal Length**: The distance between the camera lens and the sensor.

Smaller GSD means higher spatial resolution, as each pixel covers a smaller area on the ground.

In [20]:
# Calculate the GSD (Ground Sample Distance)
gsd = (altitude * pixel_pitch) / (focal_length / 1000)

# print GSD in meters and cm round to 2 decimal places
print("GSD: ", round(gsd, 2), "meters / px")
print("GSD: ", round(gsd * 100, 2), "cm / px")


GSD:  0.02 meters / px
GSD:  2.5 cm / px


We can estimate how large a tree would look on an image in terms of pixels by adjusting our Ground Sampling Distance (GSD) calculation:

$$ \text{GSD} = \frac{(\text{altitude} - \text{tree height}) \times \text{pixel pitch}}{\text{focal length}} $$

In Vancouver, BC, Canada, the average tree's height is about 10 meters and the average tree's cannopy diameter is also about 10 meters.

In [21]:
import numpy as np

height_tree = 10  # meters

# GSD at the tree height
gsd = (altitude - height_tree) * pixel_pitch / focal_length

diameter_of_tree = 10  # meters

# Number of pixels the tree will cover
diameter_of_tree_px = diameter_of_tree / gsd

print("Area of tree: ", int(diameter_of_tree_px), "pixels")

Area of tree:  441394 pixels
